In [1]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from components import Components
import os
load_dotenv()

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
rag_components = Components("Snowflake/snowflake-arctic-embed-s","mixedbread-ai/mxbai-embed-large-v1","mixtral-8x7b-32768")

In [15]:
llm = rag_components.get_groq_llm(system_prompt="""
You will be given a user_question, ground_truth, system_answer couple.
Your task is to provide a 1 or 0 scoring how well the system_answer answers the user concerns expressed in the user_question in relation to the ground truth.
Give your answer as a 1 or 0, where 0 means that the system_answer is not helpful at all, and 1 means that the answer completely correctly, accurately and helpfully addresses the question.

Provide your answer directly, you may NOT write anything else besides 1 or 0.

Now here are the question and answer.

user_question: {question}
ground_truth : {answer}                                  
system_answer: {gen_answer}

 """)

In [8]:
df_qa = pd.read_csv('../data/MainDataset/results/llama3-8b-8192_RAG_Run1.csv')

In [16]:
import time
response_list = []
for i in range(len(df_qa)): 
    time.sleep(5)
    question = """
    user_question: {}
    ground_truth : {}
    system_answer: {}
    """.format(df_qa['question'][i], df_qa['answers'][i], df_qa['gen_answer'][i])
    print(question)
    response = llm.complete(question)
    print(response)
    rowIndex = df_qa.index[i]
    df_qa.at[rowIndex, 'truth_llm'] = response.text
    print('')
    print("--"*100)
    print(f'{i}/{128}')
 



    user_question: who introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands? brainly
    ground_truth : Joseph Lister
    system_answer: According to the context, Joseph Lister introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands, with his paper "Antiseptic Principle of the Practice of Surgery" in 1867.
    
1

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0/128

    user_question: what kind of government was chiang kai-shek trying to build in china
    ground_truth : Chiang Kai-Shek was trying to build a Communist government in China.
    system_answer: According to the context, Chiang Kai-shek was trying to build a Nationalist government in China. He was the leader of the Kuomintang, the nationalist movement in China, and aimed to unify China and fight c

In [17]:
df_qa.to_csv('../data/MainDataset/results/llama3-8b-8192_RAG_Run1_LLMASAJUDGE.csv', index=False)